# Text Classification
* 적용사례 : 감정분석(긍부정) , 스팸필터링

In [ ]:
# microsoft visual c++ build tool 설치
# ! pip install jpype1
# ! pip install konlpy
# ! pip install customized_konlpy

In [ ]:
# Warning Off

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
from konlpy.tag import Okt, Kkma, Hannanum
okt = Twitter()
kkma = Kkma()
hannanum = Hannanum()
# mecab = Mecab() # mecab은 윈도우에서 사용불가

# 입력데이터

In [ ]:
df = pd.read_csv('sample.csv', encoding='cp949')

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.shape

# 분류현황

In [ ]:
category_count = df['대상'].value_counts()
category_count

# null 제거

In [ ]:
df_bak = df.copy()

In [ ]:
df = df.fillna('N') # null 값을 N으로 채움

# 전처리

In [ ]:
# 예측값과 실제값 비교를 위해 컬럼 하나 추가 파생
df['target_origin'] = df['대상'].astype(str).copy() 

In [ ]:
def preprocessing(text):
    text = str(text)
    text = re.sub('\\\\n', ' ', text) # 개행문자 제거
    text = re.sub('?.,;:|\)*~(_+=<>!@#$%^&*※', ' ', text) # 특수문자 제거
    text = re.sub('[가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text) # 한글, 영문, 숫자 외에는 제거
    return text    

In [ ]:
%time df['text'] = df['text'].apply(preprocessing)

# 학습데이터와 테스트셋 생성

In [ ]:
test_split

In [ ]:
df_train = df[:int(df.shape[0])*0.7].copy()
df_test = df[int(df.shape[0])*0.7:].copy()

# 단어 벡터화

In [ ]:
from sklearn.feature_extraction.text import CountVectrizer

stop_words_list = ['kk','dd']
vectorizer = CountVectorizer(analyzer = 'word' # character 단위로도 벡터화 가능 
                            tokenizer = None # Tokenizer 지정 가능
                            stop_words = stop_words_list # 불용어 nltk등의 도구 사용가능 
                            min_df = 1, # 토큰이 나타날 최소 문서 개수로 오타나 자주 나오지 않는 특수한 전문용어 제거에 좋음
                            ngram_range = (1,3) # BOW 단위 지정 1~3개
                            max_features = 20000 # 만들 피처수(단어의 수) 
                            )
vectorizer

In [ ]:
%%time
train_feature_vector = vectorizer.fit_transform(df_train['text'])
train_feature_vector.shape

In [ ]:
%%time
test_feature_vector = vectorizer.fit_transform(df_test['text'])
test_feature_vector.shape

In [ ]:
vocab = vectorizer.get_feature_names()
print(len(vocab))
vocab[:10]

In [ ]:
dist = np.sum(train_feature_vector, axis=0)

pd.Dataframe(dist, columns=vocab)

# df-idf 가중치 적용

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)
transformer

In [ ]:
$$time
train_feature_tfidf = transform(train_feature_vector)
test_feature_tfidf.shape

# 머신러닝 : 학습

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# 랜덤포레스트 분류기 사용
forest = RandomForestClassifier(
    n_estimators = 100, n_jobs = -1, random_state=2019)
forest

In [ ]:
# 학습에 사용할 y_label을 넣어줌
# 어떤 카테고리에 대한 예측이므로 category를 넣어줍니다
y_label = df_train['target_pred']
%time forest = forest.fit(train_feature_tfidf, y_label)

# 모델평가

In [ ]:
from sklearn.model_selection import KFold, cross_val_score

k_fold = KFold(n_splits=5, shuffle=True, random_state=2019)
scoring = 'accuracy'
score = cross_val_score(forest, train_feature_vector, y_label, cv=k_fold, n_jobs=-1, scoring=scoring)
score

In [ ]:
round(np.mean(score)*100)

# 예측

In [ ]:
# 테스트 데이터를 넣고 예측

y_pred = forest.predict(test_feture_vector)
y_pred[:3]

In [ ]:
y_pred.shape

In [ ]:
# 예측결과를 저장하기 위해 데이터프레임에 담아줌

output = pd.DataFrame(data={'category_pred':y_red})
output.head(2)

# 결과검증

In [ ]:
df_test['pred_diff'] = 0
df_test['pred_diff'] = (df_test['대상 선정'] = df_test['category_pred']) == 1
df_test['pred_diff'] = df_test['pred_diff'].astype(int)
df_test.head(2)

In [ ]:
df_test['pred_diff'].sum()

# 정확도

In [2]:
899/1000

0.899